In [ ]:
import yfinance as yf
import pandas as pd
import datetime as dt
import pandas_datareader.data as web

from yahooquery import Ticker

import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import datetime as dt
import plotly.graph_objects as go

import pandas as pd
from openpyxl import load_workbook

In [ ]:
start = dt.datetime(2024, 1, 1)
end = dt.datetime(2025, 4, 13) #dt.datetime.now()

# Diccionario con nombres de índices y sus símbolos de Yahoo Finance
us_indices = {
    'S&P 500': '^GSPC',
    'Nasdaq': '^IXIC',
    'Dow Jones': '^DJI',
    'Russell 2000': '^RUT'
}

europe_indices = {
    'FTSE 100': '^FTSE',
    'DAX': '^GDAXI',
    'CAC 40': '^FCHI',
    'Euro Stoxx 50': '^STOXX50E',
    'Euronext 100': '^N100'
}

latam_indices = {
    'Bovespa': '^BVSP',
    # 'IPSA (Chile)': '^IPSA',
    'COLCAP (Colombia)': '^COLCAP',
    # 'S&P/BVL General (Peru)': '^SPBLPGPT',
    'MSCI Peru ETF': 'EPU'
}

asia_indices = {
    'Nikkei 225 (Japan)': '^N225',
    'Hang Seng (Hong Kong)': '^HSI',
    'Shanghai Composite (China)': '000001.SS',
    'Kospi (South Korea)': '^KS11'
}

# Función para descargar y normalizar datos
# def get_normalized_data(indices, start, end):
#     dataframes = []
#     for name, ticker in indices.items():
#         df = yf.download(ticker, start=start, end=end)[['Close']]
#         time.sleep(5)
#         df.rename(columns={'Close': name}, inplace=True)
#         df = df.dropna(subset=[name])
#         if not df.empty:
#             df[name] = df[name] / df[name].iloc[0] * 100  # Normalizar a base 100
#             dataframes.append(df)
#     return dataframes

def get_normalized_data(indices, start, end):
    tickers = list(indices.values())
    tq = Ticker(tickers)
    data = tq.history(start=start.strftime('%Y-%m-%d'), end=end.strftime('%Y-%m-%d'))

    if isinstance(data.index, pd.MultiIndex):
        data = data.reset_index()

    dataframes = []
    for name, ticker in indices.items():
        df = data[data['symbol'] == ticker][['date', 'close']]
        df.set_index('date', inplace=True)
        df.rename(columns={'close': name}, inplace=True)
        df = df.dropna()
        if not df.empty:
            df[name] = df[name] / df[name].iloc[0] * 100  # Normalizar a base 100
            dataframes.append(df)
    return dataframes

# Descargar y procesar datos para cada región
us_dataframes = get_normalized_data(us_indices, start, end)
europe_dataframes = get_normalized_data(europe_indices, start, end)
latam_dataframes = get_normalized_data(latam_indices, start, end)
asia_dataframes = get_normalized_data(asia_indices, start, end)

# Función para combinar DataFrames
def merge_dataframes(dataframes):
    if dataframes:
        merged_df = dataframes[0]
        for df in dataframes[1:]:
            merged_df = pd.merge(merged_df, df, left_index=True, right_index=True, how='outer')
        return merged_df.fillna(method='ffill').fillna(method='bfill')
    else:
        return pd.DataFrame()

# Combinar DataFrames de cada región
us_merged = merge_dataframes(us_dataframes)
europe_merged = merge_dataframes(europe_dataframes)
latam_merged = merge_dataframes(latam_dataframes)
asia_merged = merge_dataframes(asia_dataframes)

# Función para graficar los índices de una región
def plot_indices(dataframe, title):
    fig = go.Figure()
    for column in dataframe.columns:
        fig.add_trace(go.Scatter(
            x=dataframe.index,
            y=dataframe[column],
            mode='lines',
            name=column
        ))
    fig.update_layout(
        title=title,
        xaxis_title='Fecha',
        yaxis_title='Índice (Base 100)',
        hovermode='x unified'
    )
    fig.show()

# Mostrar gráficos para cada región
plot_indices(us_merged, 'Índices de Estados Unidos (Base 100)')
plot_indices(europe_merged, 'Índices de Europa (Base 100)')
# plot_indices(latam_merged, 'Índices de América Latina (Base 100)')
plot_indices(asia_merged, 'Índices de Asia (Base 100)')


asia = asia_merged
europe = europe_merged
us = us_merged

asia.index.name = None
europe.index.name = None
us.index.name = None

asia_eu = asia.merge(europe, left_index=True, right_index=True, how='inner')
asia_eu_us_index = asia_eu.merge(us, left_index=True, right_index=True, how='inner')
asia_eu_us_index.columns.name = 'time' 

asia_eu_us_index.to_excel("../Stock Data Yahoo/data/asia_eu_us_index.xlsx",sheet_name="Indices")

In [ ]:


# Normalizar índices para eliminar componentes de tiempo
# asia_merged.index = asia_merged.index.normalize()
# europe_merged.index = europe_merged.index.normalize()
# us_merged.index = us_merged.index.normalize()

asia_merged.index = pd.to_datetime(asia_merged.index).normalize()
europe_merged.index = pd.to_datetime(europe_merged.index).normalize()
us_merged.index = pd.to_datetime(us_merged.index).normalize()

# Asegurarse de que los índices no tengan nombres
asia_merged.index.name = None
europe_merged.index.name = None
us_merged.index.name = None

# Realizar el merge secuencial
asia_eu = asia_merged.merge(europe_merged, left_index=True, right_index=True, how='inner')
asia_eu_us = asia_eu.merge(us_merged, left_index=True, right_index=True, how='inner')

# Restablecer el índice y renombrarlo como 'time'
asia_eu_us.reset_index(inplace=True)
asia_eu_us.set_index("index", inplace=True)
asia_eu_us.index.name = "time"

# Exportar a una nueva hoja de un archivo de Excel existente
output_file = "merged_data.xlsx"  # Nombre del archivo de salida
sheet_name = "Asia_EU_US_Merged"  # Nombre de la nueva hoja

# # Cargar el archivo existente y agregar una nueva hoja
# with pd.ExcelWriter(output_file, engine='openpyxl', mode='a') as writer:
#     asia_eu_us.to_excel(writer, sheet_name=sheet_name)

# print("Datos exportados exitosamente a la hoja:", sheet_name)
asia_eu_us

In [ ]:
import pandas as pd
from openpyxl import load_workbook

asia = asia_merged
europe = europe_merged
us = us_merged

asia.index.name = None
europe.index.name = None
us.index.name = None

asia_eu = asia.merge(europe, left_index=True, right_index=True, how='inner')
asia_eu_us_index = asia_eu.merge(us, left_index=True, right_index=True, how='inner')
asia_eu_us_index.columns.name = 'time' 

asia_eu_us_index.to_excel("../Stock Data Yahoo/data/asia_eu_us_index.xlsx",sheet_name="Indices")

# output_file = 'C:/Users/HP SUPPORT/Documents/GitHub/indicators/data/macro_dataset.xlsx'
# # output_file = '../Documents/GitHub/indicators/data/macro_dataset.xlsx'
# sheet_name = "Indices"
# asia_eu_us_index

# with pd.ExcelWriter(output_file, engine='openpyxl', mode='a') as writer:
#     asia_eu_us_index.to_excel(writer, sheet_name=sheet_name)

asia_eu_us_index


In [ ]:
print(asia.index.equals(europe.index))  # Debe ser True
print(asia.index.equals(us.index))     # Debe ser True


In [ ]:
import yfinance as yf
import pandas as pd
import datetime as dt
import plotly.graph_objects as go

# Fechas de inicio y fin
start = dt.datetime(2024, 1, 1)
end = dt.datetime.now()

# Diccionario con nombres de índices y sus símbolos de Yahoo Finance
# latam_indices = {
#     'Bovespa': '^BVSP',
#     'IPC México': '^MXX',
#     'Merval': '^MERV',
#     'S&P/BVL General': '^SPBLPGPT',
#     'MSCI Peru ETF': 'EPU'
# }
latam_indices = {
    'Bovespa': '^BVSP',  # Brasil
    'IPC México': '^MXX', #Mexico
    'IPSA (Chile)': '^IPSA',  # Chile
    'COLCAP (Colombia)': '^COLCAP',  # Colombia
    'S&P/BVL General (Peru)': '^SPBLPGPT',  # Perú
    'MSCI Peru ETF': 'EPU'  # ETF de Perú como representación del mercado
}


# Descargar y preparar datos de América Latina
latam_dataframes = []
for name, ticker in latam_indices.items():
    df = yf.download(ticker, start=start, end=end)[['Close']]
    df.rename(columns={'Close': name}, inplace=True)
    
    # Eliminar NaN al inicio
    df = df.dropna(subset=[name])
    
    # Verificar si el DataFrame no está vacío antes de la normalización
    if not df.empty:
        df[name] = df[name] / df[name].iloc[0] * 100  # Normalizar a base 100
        latam_dataframes.append(df)

# Verificar que la lista de DataFrames no esté vacía antes del merge
if latam_dataframes:
    latam_merged = latam_dataframes[0]
    for df in latam_dataframes[1:]:
        latam_merged = pd.merge(latam_merged, df, left_index=True, right_index=True, how='outer')


    latam_merged = latam_merged.dropna()


    # Crear gráfico para América Latina
    fig = go.Figure()
    for column in latam_merged.columns:
        fig.add_trace(go.Scatter(
            x=latam_merged.index,
            y=latam_merged[column],
            mode='lines',
            name=column
        ))
    fig.update_layout(
        title='Índices de América Latina (Base 100)',
        xaxis_title='Fecha',
        yaxis_title='Índice (Base 100)',
        hovermode='x unified'
    )
    fig.show()
else:
    print("No hay datos válidos para los índices de América Latina.")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Función para obtener los datos de un índice específico
def get_investing_data(index_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # Realizamos la solicitud GET al sitio web
    response = requests.get(index_url, headers=headers)
    
    # Si la solicitud fue exitosa
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encontramos la tabla de datos históricos (dependiendo de la estructura del HTML)
        table = soup.find('table', {'class': 'genTbl closedTbl historicalTbl'})
        
        # Extraemos las filas de la tabla (cada fila es un día de datos)
        rows = table.find_all('tr')
        
        # Creamos listas para almacenar los datos
        dates = []
        opens = []
        highs = []
        lows = []
        closes = []
        volumes = []
        
        # Iteramos sobre cada fila de datos
        for row in rows[1:]:  # Saltamos la primera fila que es el encabezado
            cols = row.find_all('td')
            if len(cols) > 1:  # Si la fila tiene datos
                dates.append(cols[0].text.strip())
                opens.append(cols[1].text.strip())
                highs.append(cols[2].text.strip())
                lows.append(cols[3].text.strip())
                closes.append(cols[4].text.strip())
                volumes.append(cols[5].text.strip())
        
        # Convertimos las listas en un DataFrame de pandas
        data = {
            'Date': dates,
            'Open': opens,
            'High': highs,
            'Low': lows,
            'Close': closes,
            'Volume': volumes
        }
        
        df = pd.DataFrame(data)
        
        # Convertir las fechas al formato correcto (puedes ajustarlo según la región)
        df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y')
        
        return df
    else:
        print("Error al obtener los datos")
        return None

# URL del índice S&P Merval (puedes cambiar la URL para otros índices)
url = 'https://www.investing.com/indices/sp-bvlmerval-historical-data'

# Obtener los datos
data = get_investing_data(url)

# Mostrar los primeros registros
if data is not None:
    print(data.head())

    # Guardar los datos en un archivo CSV
    data.to_csv('sp_merval_data.csv', index=False)


### LATAM Y CARIBE

IPSA CHILE: https://es.investing.com/indices/ipsa-historical-data

S&P LIMA GENERAL: https://es.investing.com/indices/lima-stock-exchange-general

COLCAP: https://es.investing.com/indices/colcap

S&P/BMV IPC (MXX): https://es.investing.com/indices/ipc

S&P MERVAL: https://es.investing.com/indices/merv


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Configura el navegador
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # si no quieres que se abra la ventana
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL del índice S&P Lima General
url = "https://es.investing.com/indices/lima-stock-exchange-general-historical-data"
driver.get(url)
time.sleep(5)  # espera para que cargue el contenido dinámico

# Aceptar cookies si aparece
try:
    cookie_btn = driver.find_element(By.XPATH, '//button[contains(text(), "Aceptar")]')
    cookie_btn.click()
    time.sleep(2)
except:
    pass

# Obtener fecha del rango visible
fecha_rango = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[2]/div[1]/div[2]/div[2]/div[2]/div[2]/div').text
print("Rango de fechas:", fecha_rango)

# Obtener la tabla
tabla = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[2]/div[1]/div[2]/div[3]/table/tbody')
filas = tabla.find_elements(By.TAG_NAME, 'tr')

# Parsear datos
datos = []
for fila in filas:
    columnas = fila.find_elements(By.TAG_NAME, 'td')
    datos.append([col.text for col in columnas])

# Convertir a DataFrame
columnas = ['Fecha', 'Último', 'Apertura', 'Máximo', 'Mínimo', 'Volumen', '% Var']
df = pd.DataFrame(datos, columns=columnas)

# Convertir columnas numéricas
df['Último'] = df['Último'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)
df['% Var'] = df['% Var'].str.replace('%', '').str.replace(',', '.', regex=False).astype(float)
df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True)

# Mostrar
print(df.head())

driver.quit()

# FECHA INICIO: //*[@id="__next"]/div[2]/div[2]/div[2]/div[1]/div[2]/div[2]/div[2]/div[3]/div[1]/div[1]/input
# FECHA FIN: //*[@id="__next"]/div[2]/div[2]/div[2]/div[1]/div[2]/div[2]/div[2]/div[3]/div[1]/div[2]/input
# ACEPTAR : //*[@id="__next"]/div[2]/div[2]/div[2]/div[1]/div[2]/div[2]/div[2]/div[3]/div[2]/span[2]

Rango de fechas: 17.03.2025 - 18.04.2025
       Fecha    Último   Apertura     Máximo     Mínimo Volumen  % Var
0 2025-04-15  29635.17  29.725,68  29.971,63  29.618,31          -0.30
1 2025-04-14  29725.68  29.450,70  29.828,94  29.446,71           0.91
2 2025-04-13  29458.58  29.014,29  29.495,22  29.014,29           1.53
3 2025-04-10  29013.90  28.485,83  29.137,22  28.485,83           1.84
4 2025-04-09  28489.65  28.684,31  28.765,84  28.258,39          -0.68


In [5]:
df

,Fecha,Último,Apertura,Máximo,Mínimo,Volumen,% Var
0,2025-04-15,29635.17,"29.725,68","29.971,63","29.618,31",,-0.30
1,2025-04-14,29725.68,"29.450,70","29.828,94","29.446,71",,0.91
2,2025-04-13,29458.58,"29.014,29","29.495,22","29.014,29",,1.53
3,2025-04-10,29013.90,"28.485,83","29.137,22","28.485,83",,1.84
4,2025-04-09,28489.65,"28.684,31","28.765,84","28.258,39",,-0.68
5,2025-04-08,28684.31,"27.972,13","28.812,86","27.972,13",,2.54
6,2025-04-07,27974.37,"27.841,22","28.378,02","27.841,22",,0.48
7,2025-04-06,27840.85,"28.158,26","28.214,46","26.745,57",,-1.13
8,2025-04-03,28157.88,"29.560,23","29.561,52","28.157,88",,-4.76
9,2025-04-02,29564.26,"30.216,47","30.216,47","29.556,68",,-2.15
